In [1]:
import pandas as pd
import numpy as np

C:\Users\paulo\AppData\Local\Temp\ipykernel_12132\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Read the Excel file
excel_file = "C:\\Users\\paulo\\Downloads\\Erdos_data_with clamp.xlsx"

In [3]:
output_file = 'C:\\Users\\paulo\\Downloads\\transformed_data.csv'

In [4]:
def process_ogtt_sheet(file_name, sheet_name, metab):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    df_melted = df.melt(id_vars=['ID'], var_name='time', value_name='VAL')
    df_melted = df_melted.sort_values(['ID', 'time'])
    df_melted['ID'] = df_melted['ID'].str.replace('PS', 'Ex')
    df_melted['Condition'] = 'na'
    df_melted['test'] = 'OGTT'
    df_melted['metab'] = metab
    return df_melted[['ID', 'Condition', 'test', 'metab', 'VAL', 'time']]

In [5]:
def process_cgm_sheet(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    df_melted = df.melt(id_vars=['ID'], var_name='time', value_name='VAL')
    df_melted = df_melted.sort_values(['ID', 'time'])
    df_melted['ID'] = df_melted['ID'].str.replace('PS', 'Ex')
    df_melted['Condition'] = 'na'
    df_melted['test'] = 'CGM'
    df_melted['metab'] = 'gluc'
    return df_melted[['ID', 'Condition', 'test', 'metab', 'VAL', 'time']]

In [6]:
# Process OGTT glucose data
df_ogtt_glucose = process_ogtt_sheet(excel_file, 'OGTT_week1_plasma_glucose', 'gluc')

# Process OGTT insulin data
df_ogtt_insulin = process_ogtt_sheet(excel_file, 'OGTT_week1_plasma_insulin', 'ins')

# Process CGM glucose data
df_cgm_glucose = process_cgm_sheet(excel_file, 'OGTT_week1_CGM_glucose')

# Combine all data
df_final = pd.concat([df_ogtt_glucose, df_ogtt_insulin, df_cgm_glucose], ignore_index=True)

# Sort the final dataframe
df_final = df_final.sort_values(['ID', 'test', 'metab', 'time'])

# Remove rows with NaN values
df_final = df_final.dropna()

# Save to CSV
df_final.to_csv(output_file, index=False)

print(df_final.head(20))
print("\nData shape:", df_final.shape)

# Additional data validation
print("\nUnique IDs:")
print(df_final['ID'].nunique())
print("\nID counts:")
print(df_final['ID'].value_counts())
print("\nTime points per ID, test, and metabolite:")
print(df_final.groupby(['ID', 'test', 'metab'])['time'].nunique())
print("\nValue ranges:")
print(df_final.groupby(['test', 'metab'])['VAL'].describe())


            ID Condition test metab  VAL time
3402  Ex010001        na  CGM  gluc  4.6    0
3403  Ex010001        na  CGM  gluc  4.8    5
3404  Ex010001        na  CGM  gluc  5.2   10
3405  Ex010001        na  CGM  gluc  5.7   15
3406  Ex010001        na  CGM  gluc  6.3   20
3407  Ex010001        na  CGM  gluc  6.9   25
3408  Ex010001        na  CGM  gluc  7.4   30
3409  Ex010001        na  CGM  gluc  7.8   35
3410  Ex010001        na  CGM  gluc  8.0   40
3411  Ex010001        na  CGM  gluc  8.0   45
3412  Ex010001        na  CGM  gluc  8.0   50
3413  Ex010001        na  CGM  gluc  7.9   55
3414  Ex010001        na  CGM  gluc  7.8   60
3415  Ex010001        na  CGM  gluc  7.7   65
3416  Ex010001        na  CGM  gluc  7.5   70
3417  Ex010001        na  CGM  gluc  7.4   75
3418  Ex010001        na  CGM  gluc  7.3   80
3419  Ex010001        na  CGM  gluc  7.2   85
3420  Ex010001        na  CGM  gluc  7.0   90
3421  Ex010001        na  CGM  gluc  6.8   95

Data shape: (9258, 6)

Unique IDs